<a href="https://colab.research.google.com/github/rafaelpierre/mistral-7b-finetune-nf4/blob/main/Mistral_7b_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine Tuning Mistral-7b
### Using Google Collab and Nvidia T4 GPU

### Step 1: Installing Dependencies

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q bitsandbytes accelerate xformers einops peft trl==0.7.5 torch==2.1.0

In [ ]:
!nvidia-smi

## Fine Tuning

### Downloading Dataset

In [ ]:
from datasets import load_dataset
import logging

logging.basicConfig(level = "DEBUG")

dataset = load_dataset("timdettmers/openassistant-guanaco")

In [ ]:
dataset

In [ ]:
dataset["train"][0]

### Converting instructions from Alpaca format to Mistral

In [ ]:
# The dataset is in the Alpaca format, we need to convert it to the format expected by Mistral

import re
from datasets import DatasetDict

MISTRAL_TEMPLATE = """<s>[INST]{instruction}[/INST]
{answer}</s>
"""

def convert_row(example, human_prefix: str = "### Human:", assistant_prefix: str = "### Assistant:"):

  try:

    instruction = re.search(f"(?<=({human_prefix}))(.|\n)+(?=({assistant_prefix}))", example['text'])[0]
    answer = re.search(f"(?<=({assistant_prefix}))(.|\n)+", example['text'])[0].split(human_prefix)[0]
    clean_answer = answer.split(human_prefix)[0]

    ds_object = {
        "text": MISTRAL_TEMPLATE.format(
          instruction = instruction,
          answer = clean_answer
        )
    }

    return ds_object
  except:
    print("error")

def create_mistral_dataset(dataset):

  mistral_dataset = dataset.map(convert_row)
  return mistral_dataset

mistral_dataset = create_mistral_dataset(dataset)


In [ ]:
mistral_dataset["train"][0]

In [ ]:
new_model = "mistralai-openassistant-guanaco"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 2

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 3

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 2048

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
import torch
from transformers import (
  AutoModelForCausalLM,
  AutoTokenizer,
  BitsAndBytesConfig,
  pipeline
)

from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load the base model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype = torch.float16,
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage = True
)

base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

# Load MitsralAi tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Set LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj"
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

#Prepare model for finetuning

base_model = prepare_model_for_kbit_training(base_model)
base_model = get_peft_model(base_model, peft_config)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=100, # the total number of training steps to perform
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Initialize the SFTTrainer for fine-tuning
trainer = SFTTrainer(
    model=base_model,
    train_dataset=mistral_dataset["train"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,  # You can specify the maximum sequence length here
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

In [ ]:
# Start the training process
trainer.train()

In [ ]:
from huggingface_hub import HfApi

token = input("Please enter your HF Token: ")
api = HfApi(token = token)

In [ ]:
api.create_repo(repo_id = "mistral-7b-instruct-4bit-openassistant-guanaco")

In [ ]:
base_model.push_to_hub("mistral-7b-instruct-4bit-openassistant-guanaco", use_auth_token = token)